In [1]:
import json
import psycopg2

In [2]:
# Connect to the postgres server linked to the container this notebook is running in
connection = psycopg2.connect("host=postgres dbname=wisdom user=postgres")

In [3]:
# Query the first variant from every report and print its gene
with connection.cursor() as cursor:
    cursor.execute("select report->'variants'->1 from reports")
    variants = cursor.fetchall()
    print("Found variants for", len(variants), "patients")
    print([v[0]["gene"] for v in variants])

Found variants for 250 patients
['MSH2', 'BRCA1', 'BRCA2', 'BRCA1', 'PTEN', 'PTEN', 'ATM', 'BRCA2', 'ATM', 'MSH2', 'ATM', 'MSH6', 'BRCA1', 'PTEN', 'ATM', 'CDH1', 'PMS2', 'BRCA1', 'MLH1', 'ATM', 'PALB2', 'BRCA1', 'BRCA2', 'BRCA1', 'ATM', 'PALB2', 'PTEN', 'PALB2', 'CDH1', 'BRCA1', 'BRCA2', 'PMS2', 'BRCA1', 'ATM', 'BRCA1', 'ATM', 'PTEN', 'ATM', 'MSH6', 'CDH1', 'PMS2', 'ATM', 'ATM', 'BRIP1', 'BRCA1', 'BRCA1', 'PMS2', 'BRCA2', 'BRCA1', 'PALB2', 'BRCA1', 'ATM', 'MLH1', 'ATM', 'MSH6', 'MLH1', 'ATM', 'BRCA1', 'PALB2', 'ATM', 'ATM', 'BRCA1', 'PMS2', 'PALB2', 'PMS2', 'BRCA1', 'PALB2', 'ATM', 'MLH1', 'ATM', 'PTEN', 'BRCA1', 'BRCA1', 'MSH6', 'PTEN', 'BRCA2', 'BRCA1', 'PTEN', 'MSH6', 'ATM', 'ATM', 'MSH2', 'PALB2', 'MLH1', 'MLH1', 'PALB2', 'BRCA1', 'ATM', 'BRCA2', 'BRCA2', 'ATM', 'PMS2', 'PALB2', 'ATM', 'PALB2', 'BRCA1', 'BRIP1', 'BRCA2', 'BRCA2', 'ATM', 'ATM', 'BRCA2', 'ATM', 'BRCA1', 'PTEN', 'MLH1', 'ATM', 'CDH1', 'BRCA1', 'BRCA1', 'ATM', 'BRIP1', 'MSH2', 'MSH6', 'BRCA1', 'BRCA1', 'BRCA1', 'PTEN',